In [8]:
import requests
from bs4 import BeautifulSoup
#首先，一樣要先透過requests.get連到該目標網址，然後一樣丟給了BeauitfulSoup去處理!
resp = requests.get('http://jwlin.github.io/py-scraping-analysis-book/ch2/table/table.html')
soup = BeautifulSoup(resp.text, 'html.parser')

# 計算課程均價
# 取得所有課程價錢: 方法一, 使用 index
prices = []
#tr就跟row一樣，所以先取tr資料
#然後就透過迴圈去把所有tr的價格資料取出，價格td在第三欄，以index來計算的話是2。
rows = soup.find('table', 'table').tbody.find_all('tr')
for row in rows:
    price = row.find_all('td')[2].text
    prices.append(int(price))
print(sum(prices)/len(prices))

1823.3333333333333


In [9]:
# 取得所有課程價錢: 方法二, <a> 的 parent (<td>) 的 previous_sibling
prices = []
links = soup.find_all('a')
for link in links:
    price = link.parent.previous_sibling.text
    #.parent(父).previous_sibling(兄) 
    prices.append(int(price))
print(sum(prices) / len(prices))

1823.3333333333333


In [10]:
# 取得每一列所有欄位文字資訊: stripped_strings
rows = soup.find('table', 'table').tbody.find_all('tr')
for row in rows:
    print([s for s in row.stripped_strings])

['初心者 - Python入門', '初學者', '1490']
['Python 網頁爬蟲入門實戰', '有程式基礎的初學者', '1890']
['Python 機器學習入門實戰 (預計)', '有程式基礎的初學者', '1890']
['Python 資料科學入門實戰 (預計)', '有程式基礎的初學者', '1890']
['Python 資料視覺化入門實戰 (預計)', '有程式基礎的初學者', '1890']
['Python 網站架設入門實戰 (預計)', '有程式基礎的初學者', '1890']


In [11]:
# 取得每一列所有欄位資訊: find_all('td') or row.children
rows = soup.find('table', 'table').tbody.find_all('tr')
for row in rows:
    all_tds = row.find_all('td')  
    # 方法一: find_all('td')
    # all_tds = [td for td in row.children]  
    # 方法二: 找出 row (tr) 所有的直接 (下一層) children
    # 以下執行時會報錯, 因為最後一列的 <a> 沒有 'href' 屬性
    # print(all_tds[0].text, all_tds[1].text, all_tds[2].text, all_tds[3].a['href'], all_tds[3].a.img['src'])
    # 取得 href 屬性前先檢查其是否存在
    if 'href' in all_tds[3].a.attrs:
        href = all_tds[3].a['href']
    else:
        href = None
    print(all_tds[0].text, all_tds[1].text, all_tds[2].text, href, all_tds[3].a.img['src'])

初心者 - Python入門 初學者 1490 http://www.pycone.com img/python-logo.png
Python 網頁爬蟲入門實戰 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 機器學習入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 資料科學入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 資料視覺化入門實戰 (預計) 有程式基礎的初學者 1890 http://www.pycone.com img/python-logo.png
Python 網站架設入門實戰 (預計) 有程式基礎的初學者 1890 None img/python-logo.png


In [12]:
#attrs屬性
html = '<div data-num="123">foo!</div>'
soup = BeautifulSoup(html, 'html.parser')
data_tag = soup.find(attrs={"data-num": "123"})
print(data_tag)

<div data-num="123">foo!</div>


In [18]:
import requests
import re
import json
from bs4 import BeautifulSoup


Y_MOVIE_URL = 'https://tw.movies.yahoo.com/movie_thisweek.html'
def get_web_page(url):
    resp = requests.get(url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text
def get_date(date_str):
    # e.g. "上映日期：2017-03-23" -> match.group(0): "2017-03-23"
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)


def get_movie_id(url):
    # e.g., 'https://movies.yahoo.com.tw/movieinfo_main/%E6%AD%BB%E4%BE%8D2-deadpool-2-7820.html
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id
def get_movies(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    movies = []
    rows = soup.find_all('div', 'release_info_text')
    for row in rows:
        movie = dict()
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        movie['movie_id'] = get_movie_id(row.find('div', 'release_movie_name').a['href'])
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        movie['release_date'] = get_date(row.find('div', 'release_movie_time').text)
        movies.append(movie)
    return movies
if __name__ == '__main__':
    page = get_web_page(Y_MOVIE_URL)
    movies = get_movies(page)
    for movie in movies:
        print(movie)

{'expectation': '87%', 'ch_name': '記憶屋', 'eng_name': 'kiokuya', 'movie_id': '10470', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/January2020/V6MY1SapAqTFmjnS4ChM-800x1138.jpg', 'release_date': '2020-03-27'}
{'expectation': '89%', 'ch_name': 'Re從零開始的異世界生活 外傳集', 'eng_name': '', 'movie_id': '10480', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/January2020/TDRN5rFEzoyTnUEnQ6EN-500x720.jpg', 'release_date': '2020-03-27'}
{'expectation': '88%', 'ch_name': '第六感生死戀', 'eng_name': 'Ghost', 'movie_id': '10504', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/March2020/ygqvTlkSMVgefHs3LplV-1000x1424.jpg', 'release_date': '2020-03-27'}
{'expectation': '81%', 'ch_name': '抓住救命稻草的野獸們', 'eng_name': 'Beasts Clawing at Straws', 'movie_id': '10507', 'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/February2020/uxiNS7oVZZ92gPsGXJl5-1280x1834.jpg', 'release_date': '2020-03-27'}
{'expectation': '85%'

In [1]:
import requests
from bs4 import BeautifulSoup

def append_list_pm25():
    url = 'https://taqm.epa.gov.tw/pm25/tw/PM25A.aspx?area=1'
    html = requests.get(url)
    sp = BeautifulSoup(html.text, 'html.parser')
    rs = sp.find_all("tr", {"align": "center", "style": "border-width:1px;border-style:Solid;"})
    for r in rs:
        name = r.find('a')
        pm25 = r.find_all('span')
        dic = {}
        dic.setdefault('name',   name.text.strip())
        dic.setdefault('pm25',   pm25[0].text.strip())
        dic.setdefault('pm25_1', pm25[1].text.strip())
        list.append(dic)

def get_pm25(name):
    for d in list:
        if d.get('name') == name:
            return d

list = []
append_list_pm25()
print(list)

name = input('請輸入地區 ? (例如:林口, 桃園) : ')
d = get_pm25(name)
print(d)
print(d.get('pm25'))

[{'name': '富貴角', 'pm25': '15', 'pm25_1': '18'}, {'name': '萬里', 'pm25': '', 'pm25_1': '21'}, {'name': '淡水', 'pm25': '21', 'pm25_1': '23'}, {'name': '林口', 'pm25': '36', 'pm25_1': '30'}, {'name': '三重', 'pm25': '29', 'pm25_1': '29'}, {'name': '菜寮', 'pm25': '26', 'pm25_1': '25'}, {'name': '汐止', 'pm25': '21', 'pm25_1': '22'}, {'name': '新莊', 'pm25': '28', 'pm25_1': '27'}, {'name': '永和', 'pm25': '24', 'pm25_1': '24'}, {'name': '板橋', 'pm25': '18', 'pm25_1': '23'}, {'name': '土城', 'pm25': '34', 'pm25_1': '20'}, {'name': '新店', 'pm25': '21', 'pm25_1': '25'}, {'name': '陽明', 'pm25': '28', 'pm25_1': '26'}, {'name': '士林', 'pm25': '17', 'pm25_1': '24'}, {'name': '大同', 'pm25': '21', 'pm25_1': '22'}, {'name': '中山', 'pm25': '26', 'pm25_1': '29'}, {'name': '松山', 'pm25': '28', 'pm25_1': '25'}, {'name': '萬華', 'pm25': '23', 'pm25_1': '30'}, {'name': '古亭', 'pm25': '29', 'pm25_1': '26'}, {'name': '基隆', 'pm25': '16', 'pm25_1': '13'}, {'name': '大園', 'pm25': '27', 'pm25_1': '31'}, {'name': '觀音', 'pm25': '19', 'pm25